# Q1: Simple CNN network for PASCAL multi-label classification (20 points)
Now let’s try to recognize some natural images. We provided some starter code for this task. The following steps will guide you through the process.


## 1.1 Setup the dataset
We start by modifying the code to read images from the PASCAL 2007 dataset. The important thing to note is that PASCAL can have multiple objects present in the same image. Hence, this is a multi-label classification problem, and will have to be tackled slightly differently.


First, download the data. `cd` to a location where you can store 0.5GB of images. Then run:
```
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
tar -xf VOCtrainval_06-Nov-2007.tar

wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
tar -xf VOCtest_06-Nov-2007.tar
cd VOCdevkit/VOC2007/
```

## 1.2 Write a dataloader with data augmentation (5 pts)
**Dataloader** The first step is to write a [pytorch data loader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) which loads this PASCAL data. Browse the folders and files under `VOCdevkit` to understand the structure and labeling. Complete the functions `preload_anno` and `__getitem__` in `voc_dataset.py` according to the following instructions and the instructions in the code. More information about the dataset can be found [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/devkit_doc_07-Jun-2007.pdf). We will use data in ‘trainval’ for training and ‘test’ for testing.

- `preload_anno`: This function will be called when the dataloader is initialized. We will load the annotations under folder `Annotations`. Each .xml file in the `Annotations` folder corresponds to the image with the same name under `JPEGImages`. In this function, we need to load `label` and `weight` vectors for each image according to the .xml file. 
- The labels should be 0 by default. Assign 1 for each class label in the .xml file. For example, in 000001.xml, the label vector should have 1s at the class indices correspond to 'dog' and 'person'. The rest of the vector should be 0.
- The weights should be 1 by defatul. For each class label in the image, if 'difficult'=1 (which means it is ambiguous), we will assign 0 for the weight vector at this class index. This weight will be used when we calculate the test performance. We will not consider the ambiguous labels during testing.

- `__getitem__`: This function will be called when the dataloader is called during training. It takes as input the index, and returns a tuple - `(image, label, weight)`. You need to load the image from the `JPEGImages` folder and load the corresponding label and weight using `self.anno_list`.


**Data Augmentation** Modify `__getitem__` to randomly *augment* each datapoint using [TORCHVISION.TRANSFORMS](https://pytorch.org/vision/stable/transforms.html).  Make sure the data augmentation is only used for training data (based on self.split). Please describe what data augmentation you implement.

- Before any augmentation, resize all the images based on `self.size`.

* **Hint**: Since we are training a model from scratch on this small dataset, it is important to perform basic data augmentation to avoid overfitting. Add random crops and left-right flips when training, and do a center crop when testing, etc. As for natural images, another common practice is to subtract the mean values of RGB images from ImageNet dataset. The mean values for RGB images are: `[123.68, 116.78, 103.94]`. You may also rescale the images to `[−1, 1]`. There is no "correct" answer here! Feel free to search online about the data augmentation methods people usually use.

### DESCRIBE YOUR AUGMENTATION PIPELINE HERE

**Train Augmentations:**

- I have gone with a widelly used setup of transforms
    - RandomResizedCrop - This gets a random 224x224 patch from the input image.
    - RandomHorizontalFlip - This flips the image along the vertical axis with a probability of 0.5
    - ColorJitter - This introduces pertubations in the brightness, saturation and hue of the image.
    - ToTensor - Converts it to a torch tensor and scales the values between 0 and 1.
    - Normalize - Normalizes and recenters the RGB intensity values.
- These augmentations bring enough variation to the original dataset/image so, the network doesn't overfit easily and incorporates a bit of robustness in its learning.
<br>

**Test Augmentations:**

- The goal with the test augmentations is to ensure that the images are in the right size and scale. However, there shouldn't be any other pertubations or augmentations.
  - Resize - The image is resized to a value slightly greater than 224 x 224 (~256)
  - CenterCrop - A central patch of 224x224 is taken from this resized image.
  - ToTensor - Converts it to a torch tensor and scales the values between 0 and 1.
  - Normalize - Normalizes and recenters the RGB intensity values.


## 1.3 Measure Performance (5 pts)
To evaluate the trained model, we will use a standard metric for multi-label evaluation - [mean average precision (mAP)](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). Please implement `eval_dataset_map` in `utils.py` - this function will evaluate a model's map score using a given dataset object. You will need to make predictions on the given dataset with the model and call `compute_ap` to get average precision.


Please describe how to compute AP for each class(not mAP).

**Answer:**

For each class, the precision (Of all detected as true, how many are actually true - TP / (TP + FP)) and recall values (Of all the actual true, how many have been detected - TP / (TP + FN)) are caluculated at different threshold values. The threshold value here is the probability confidence at which a detection is considered true. These different precision and recall values are used to plot the Precision-Recall curve. The area under this curve gives the AP for that class.


## 1.4 Let's Start Training! (5 pts)
Fill out the loss function for multi-label classification in `trainer.py` and start training. In this question, you will use the model that you finished in the previous question (with proper non-linearities).

Initialize a fresh model and optimizer. Then run your training code for 5 epochs and print the mAP on test set. The resulting mAP should be around 0.24. Make sure to tune the hyperparameters.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!tar -xf VOCtrainval_06-Nov-2007.tar

!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar 
!tar -xf VOCtest_06-Nov-2007.tar


--2022-02-17 20:30:18--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar.3’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  22.5MB/s    in 21s     

2022-02-17 20:30:39 (20.7 MB/s) - ‘VOCtrainval_06-Nov-2007.tar.3’ saved [460032000/460032000]

--2022-02-17 20:30:42--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar.3’

VOCtest_06-Nov-2007 100%[=============

In [ ]:
%cd drive/MyDrive/spring22/16824/hw1

/content/drive/MyDrive/spring22/16824/hw1


In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset

# create hyperparameter argument class
# Use image size of 64x64 in Q1. We will use a default size of 224x224 for the rest of the questions.
args = ARGS(epochs=5, inp_size=64, lr=0.001, use_cuda=True, step_size=2)
print(args)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
args.batch_size = 64
args.device = cuda
args.epochs = 5
args.gamma = 0.7
args.inp_size = 64
args.log_every = 100
args.lr = 0.001
args.save_at_end = False
args.save_freq = -1
args.step_size = 2
args.test_batch_size = 1000
args.val_every = 100



In [ ]:
# initializes the model
model = SimpleCNN(num_classes=len(VOCDataset.CLASS_NAMES), inp_size=64, c_dim=3)
# initializes Adam optimizer and simple StepLR scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
# trains model using your training code and reports test map
test_ap, test_map = trainer.train(args,
                                  model,
                                  optimizer,
                                  scheduler,
                                  perform_transforms=False)
print('test map:', test_map)

Train Epoch: 0 [0 (0%)]	Loss: 0.696185
Test MAP: 0.06912077030122211
Train Epoch: 1 [100 (27%)]	Loss: 0.225295
Test MAP: 0.1667342101386896
Train Epoch: 2 [200 (53%)]	Loss: 0.213764
Test MAP: 0.2125138354596569
Train Epoch: 3 [300 (80%)]	Loss: 0.190730
Test MAP: 0.22802720779210411
test map: 0.24274916326475351


[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is an awesome visualization tool. It was firstly integrated in [TensorFlow](https://www.tensorflow.org/). It can be used to visualize training losses, network weights and other parameters.

To use TensorBoard in Pytorch, there are two options: [TensorBoard in Pytorch](https://pytorch.org/docs/stable/tensorboard.html) (for Pytorch >= 1.1.0) or [TensorBoardX](https://github.com/lanpa/tensorboardX) - a third party library. Following these links to add code in `trainer.py` to visualize the testing MAP and training loss in Tensorboard. *You may have to reload the kernel for these changes to take effect.*

Show clear screenshots of the learning curves of testing MAP and training loss for 5 epochs (batch size=20, learning rate=0.001). Please evaluate your model to calculate the MAP on the testing dataset every 100 iterations.

In [ ]:
args = ARGS(epochs=5, batch_size=20, lr=0.001, inp_size=64)
model = SimpleCNN(num_classes=len(VOCDataset.CLASS_NAMES), inp_size=64, c_dim=3)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
test_ap, test_map = trainer.train(args, model, optimizer, scheduler, model_name='model_20_001', perform_transforms=False)
print('test map:', test_map)

Train Epoch: 0 [0 (0%)]	Loss: 0.684905
Test MAP: 0.06769440696318658
Train Epoch: 0 [100 (40%)]	Loss: 0.211050
Test MAP: 0.13373112827263375
Train Epoch: 0 [200 (80%)]	Loss: 0.204792
Test MAP: 0.15840676214672533
Train Epoch: 1 [300 (20%)]	Loss: 0.210824
Test MAP: 0.17806001499975693
Train Epoch: 1 [400 (59%)]	Loss: 0.219408
Test MAP: 0.20322366567868588
Train Epoch: 1 [500 (99%)]	Loss: 0.208720
Test MAP: 0.2152693991612653
Train Epoch: 2 [600 (39%)]	Loss: 0.196203
Test MAP: 0.22711121966272732
Train Epoch: 2 [700 (79%)]	Loss: 0.237052
Test MAP: 0.22825882652675858
Train Epoch: 3 [800 (19%)]	Loss: 0.177730
Test MAP: 0.2415337900002846
Train Epoch: 3 [900 (59%)]	Loss: 0.196803
Test MAP: 0.24234465702685964
Train Epoch: 3 [1000 (98%)]	Loss: 0.186077
Test MAP: 0.24544857125469957
Train Epoch: 4 [1100 (38%)]	Loss: 0.154917
Test MAP: 0.24847886105630432
Train Epoch: 4 [1200 (78%)]	Loss: 0.171937
Test MAP: 0.25019227323943877
test map: 0.25264846468947183


**INSERT YOUR TENSORBOARD SCREENSHOTS HERE**

![image](images/simple_cnn.png)